## Dataset
- You will be using a modified version of the FairFace dataset (https://github.com/joojs/fairface). This is a set of 86,744 training face images and 10,954 validation face images. 
- In order to decrease the training time I converted all images to gray scale and resized them to 32 × 32. Each face has 3 different attributes which can be used for a classification task: race, gender, and age. All files can be found in the zip file on Canvas. The train folder contains the training images and the fairface label train.csv file contains all the label. There is a similar folder and file for the validation set.
- As the three different attributes have a different number of possible values, your final layers for each classifier will vary. For each of the networks below please attempt to classify 2 of the attributes (you can choose which).

## Imports

In [ ]:
#Tensor imports
import tensorflow as tf
from tensorflow import optimizers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard

#Pillow Imports
from PIL import Image

#Import Pandas
import pandas as pd

#Import Numpy
import numpy as np

#Sci_Kit Imports
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

#Import datetime
import datetime

## Load TensorBoard and Create Logs

In [ ]:
#%load_ext tensorboard
%reload_ext tensorboard
#log_folder = "logs"
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Task 1: Fully Connected Neural Network
1. Build a feed forward neural network with the following specifications (Test on two different tasks):
    - Hidden layer 1: 1024 neurons with hyperbolic tangent activation function in each neuron.
    - Hidden layer 2: 512 neurons, with sigmoid activation function in each of the neuron.
    - 100 neurons, with rectified linear activation function in each of the neuron.
    - Output layer: n (depending on the task) neurons representing the n classes, using the softmax activation function.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset (X−Xmin/Xmax−Xmin).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix.

In [ ]:
def createFeedFoward(inputShape, outputSize, lr):
    model = Sequential()
    model.add(layers.Dense(1024, input_shape=inputShape, activation='tanh'))
    model.add(layers.Dense(512, activation='sigmoid'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(outputSize, activation='softmax'))
    opt = optimizers.SGD(learning_rate=lr)
    model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
def createXY(imgPath, labelFile, labelType, num):
    #Create and normalize X
    X = []
    for i in range(num):
        fileName = imgPath + str(i+1) + '.jpg'
        img = Image.open(fileName)
        X.append(list(img.getdata()))
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    #Get labels
    label_df = pd.read_csv(labelFile)
    
    #Find unique labels and output size
    unique = label_df[labelType].unique()
    outputSize = len(unique)
    
    #Create Binary y arrays
    lb = LabelBinarizer(sparse_output=False)
    lb.fit(unique)
    trueLabels = list(label_df[labelType].head(num))
    y = lb.transform(trueLabels)
    
    
    return X, y, outputSize, trueLabels, unique

In [ ]:
def printResults(predictions, labels, trueLabels):
    print(len(predictions))
    print(len(predictions[1]))
    print(len(labels))
    print(len(trueLabels))
    for i in range(len(predictions)):
        print("Label: " + trueLabels[i])
        for j in range(len(labels)):
            print("{:12}: {:10.2f}%".format(labels[j], (predictions[i][j] * 100)))

In [ ]:
def getMax(values):
    maxes = [np.argmax(val) for val in values ]
    return maxes

In [ ]:
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

2023-04-05 19:22:45.592266: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-05 19:22:45.592294: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-05 19:22:45.593410: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.


In [ ]:
X_train, y_train, outputSize, trueLabels, labels = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 'age', 86744)
X_test, y_test, _ , trueLabels_test, _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 'age', 10954)

In [ ]:

model = createFeedFoward((1024,), outputSize, 0.01)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=100, callbacks=callbacks)



In [ ]:
y_true = getMax(y_test)
y_pred = getMax(model.predict(X_test))
eval = tf.keras.metrics.Accuracy()
eval.update_state(y_true, y_pred)
print('Accuracy: ', eval.result().numpy())
c_matrix = confusion_matrix(y_true, y_pred)
print(c_matrix)

In [ ]:
#printResults(model.predict(X), labels, trueLabels)
%tensorboard --logdir logs --port=6007

## Task 2: Small Convolutional Neural Network
- Build a convolutional neural network with the following specifications (Test on two different tasks):
    - Convolution layer having 40 feature detectors, with kernel size 5 x 5, and ReLU as the activation function, with stride 1 and no-padding.
    - A max-pooling layer with pool size 2x2.
    - Fully connected layer with 100 neurons, and ReLU as the activation function.
    - Output layer: n (depending on the task) neurons representing the n classes, using the softmax activation function. function for each of the 10 neurons.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix.

## Task 3: Your own Convolutional Neural Network
1. Build another convolutional neural network, where you choose all the parameters to see if you can get a higher accuracy.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix

## Task 4: Your own Convolutional Neural Network on both Tasks Simultaneously
1. Build another convolutional neural network, where you try and classify both tasks with a single network. After your flatten layer have two more fully connected layers for each “branch”. Note that in order to do so you will not be able to use the Sequential model.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix

## Task 5: Variational Auto Encoder (COSC 525 only)
1. Build a variational autoencoder with the following specifications (in this one you have a little more flexibility):
    - Should have at least two convolution layers in the encoder and 2 deconvolution layers in the decoder.
    - Latent dimension should be at least 5.
    - Loss should be either MSE or binary cross entropy.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Qualitatively evaluate your model by generating a set of faces by randomly choosing 10 latent vectors and presenting the resulting images

VAE code adapted from https://keras.io/examples/generative/vae/

In [58]:
# Sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [59]:
# Encoder model
latent_dim = 12

encoder_inputs = keras.Input(shape=(32, 32, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 1)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 16, 16, 32)   320         ['input_3[0][0]']                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 8, 8, 64)     18496       ['conv2d_2[0][0]']               
                                                                                                  
 flatten_1 (Flatten)            (None, 4096)         0           ['conv2d_3[0][0]']               
                                                                                            

In [60]:
# Decoder model
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(2 * 2 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((2, 2, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 12)]              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              13312     
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 64)          0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 8, 8, 64)         36928     
 ranspose)                                                       
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 16, 16, 64)       36928     
 ranspose)                                                       
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 32, 32, 32)       1846

In [61]:
# Variational AutoEncoder model class
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [62]:
# Create and train VAE on all face images
X_train, _, _, _, _ = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 'age', 8)
X_test, _, _ , _, _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 'age', 2)
all_faces = np.concatenate([X_train, X_test], axis=0)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005))
vae.fit(np.reshape(all_faces, (10, 32, 32, 1)), epochs=10000, batch_size=10)

Epoch 1/10000
1/1 [==============================] - 1s 566ms/step - loss: 76.3538 - reconstruction_loss: 76.3441 - kl_loss: 0.0096
Epoch 2/10000
1/1 [==============================] - 0s 64ms/step - loss: 75.1152 - reconstruction_loss: 75.0668 - kl_loss: 0.0484
Epoch 3/10000
1/1 [==============================] - 0s 64ms/step - loss: 73.2715 - reconstruction_loss: 73.2615 - kl_loss: 0.0100
Epoch 4/10000
1/1 [==============================] - 0s 62ms/step - loss: 70.6400 - reconstruction_loss: 70.6393 - kl_loss: 7.3826e-04
Epoch 5/10000
1/1 [==============================] - 0s 60ms/step - loss: 68.4109 - reconstruction_loss: 68.4107 - kl_loss: 1.7102e-04
Epoch 6/10000
1/1 [==============================] - 0s 58ms/step - loss: 70.5529 - reconstruction_loss: 70.5526 - kl_loss: 3.0742e-04
Epoch 7/10000
1/1 [==============================] - 0s 63ms/step - loss: 70.2041 - reconstruction_loss: 70.2038 - kl_loss: 3.1221e-04
Epoch 8/10000
1/1 [==============================] - 0s 61ms/step 

In [68]:
# Test generating some (non-random) faces
preds = vae.encoder.predict(np.reshape(all_faces, (10, 32, 32, 1)))[0]
# gen = vae.decoder.predict(np.reshape(np.mean(preds, axis=0), (1, 5)))[0]
gen = vae.decoder.predict(np.reshape(preds[2], (1, 12)))[0]
gen = np.reshape(gen, (32, 32))
im = Image.fromarray(np.uint8(gen * 255), 'L')
im.save("test.png")

1/1 [==============================] - 0s 19ms/step
